# 01 – Data Preprocessing: Weather and Solar Data

## Objective
Prepare a clean, aggregated dataset of daily national weather metrics for use in solar generation prediction.

> **Note:** Since aggregated national-level weather data for Ireland is not publicly available, this project manually constructs a representative national dataset by averaging data from **nine strategically selected weather stations**. These stations are geographically distributed across the Republic of Ireland in a grid-like pattern to ensure balanced regional coverage.


## 1: Load Raw Station Data

- Load 9 weather station files from `Solarcast/Data/All_Weather_Stations/`
- Preview shape and content to check for structure consistency


In [16]:
import pandas as pd
import glob

csv_files = glob.glob("../data/All_Weather_Stations/*.csv")
files_shapes = []
raw_dfs = []

for file in csv_files:
    df = pd.read_csv(file)
    raw_dfs.append(df)
    files_shapes.append((file, df.shape[0], df.shape[1]))

# Convert list of tuples to DataFrame for nicer display
summary_df = pd.DataFrame(files_shapes, columns=["File", "Rows", "Columns"])
summary_df

,File,Rows,Columns
0,../data/All_Weather_Stations\Ballyhaise_Daily_...,366,24
1,../data/All_Weather_Stations\Claremorris_Daily...,366,24
2,../data/All_Weather_Stations\Dunsay_Daily_Weat...,366,24
3,../data/All_Weather_Stations\Finner_Daily_Weat...,366,24
4,../data/All_Weather_Stations\Gurteen_Daily_Wea...,366,24
5,../data/All_Weather_Stations\JohnstownCastle_D...,366,24
6,../data/All_Weather_Stations\MoorePark_Daily_W...,366,24
7,../data/All_Weather_Stations\Mullingar_Daily_W...,366,24
8,../data/All_Weather_Stations\SherkinIsland_Dai...,366,24


## 2: Standardise & Clean Columns
- Ensure uniform column names: date, rain, maxtp, mintp, cbl, glorad
- Parse dates correctly
- Handle missing or invalid entries

In [17]:
def clean_station_df(df):
    # Lowercase columns
    df.columns = df.columns.str.lower()
    
    # Filter only needed columns
    needed_cols = ['date', 'rain', 'maxtp', 'mintp', 'cbl', 'glorad']
    
    # Check what columns are present among needed columns
    available_cols = [col for col in needed_cols if col in df.columns]
    
    # Select only available columns
    df = df[available_cols].copy()
    
    # Convert 'date' to datetime if present
    if 'date' in df.columns:
        df['date'] = pd.to_datetime(df['date'], dayfirst=True, errors='coerce')
    
    # Drop rows with missing 'date' or any required weather data columns
    df = df.dropna(subset=available_cols)
    
    return df


# Clean each dataframe using clean_station_df() function. keep only non-empty ones
cleaned_dfs = []
for i, df in enumerate(raw_dfs):
    clean_df = clean_station_df(df)
    if not clean_df.empty:
        cleaned_dfs.append(clean_df)
    else:
        print(f"Station {i} cleaned DataFrame is empty or missing required data.")



C:\Users\Matthieu\AppData\Local\Temp\ipykernel_54132\2824156478.py:16: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date'] = pd.to_datetime(df['date'], dayfirst=True, errors='coerce')
C:\Users\Matthieu\AppData\Local\Temp\ipykernel_54132\2824156478.py:16: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date'] = pd.to_datetime(df['date'], dayfirst=True, errors='coerce')
C:\Users\Matthieu\AppData\Local\Temp\ipykernel_54132\2824156478.py:16: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date'] = pd.to_datetime(df['date'], dayfirst=True, errors='coerce')
C:\Users\Matt

## 3: Merge Stations by Date
- Outer join all stations on date
- Average numeric columns across all stations

In [29]:
from functools import reduce

# Station names in the same order as cleaned_dfs list
station_names = [
    'Ballyhaise', 'Claremorris', 'Dunsay', 'Finner', 'Gurteen',
    'JohnstownCastle', 'MoorePark', 'Mullingar', 'SherkinIsland'
]

# Rename columns in each cleaned DataFrame to add station suffixes (except 'date')
for i, df in enumerate(cleaned_dfs):
    suffix = f"_{station_names[i]}"
    df.rename(columns={col: col + suffix for col in df.columns if col != 'date'}, inplace=True)

# Merge all DataFrames on 'date' using outer join
merged_df = reduce(lambda left, right: pd.merge(left, right, on='date', how='outer'), cleaned_dfs)

# Calculate average of each weather parameter across stations
params = ['rain', 'maxtp', 'mintp', 'cbl', 'glorad']

avg_data = pd.DataFrame()
avg_data['date'] = merged_df['date']

for param in params:
    # Select columns for this param with station suffixes
    param_cols = [col for col in merged_df.columns if col.startswith(param + "_")]

    # Convert these columns to numeric, coercing errors to NaN
    merged_df[param_cols] = merged_df[param_cols].apply(pd.to_numeric, errors='coerce')

    # Calculate row-wise mean, skipping NaNs. Round to 2 decimal places
    avg_data[param] = merged_df[param_cols].mean(axis=1).round(2)

# Step 4: Sort by date and reset index for clean output
avg_data = avg_data.sort_values('date').reset_index(drop=True)

# Preview the averaged DataFrame
avg_data.head()


,date,rain,maxtp,mintp,cbl,glorad
0,2024-01-01,11.93,10.57,3.03,981.90,64.44
1,2024-01-02,5.77,10.22,6.53,973.32,125.44
2,2024-01-03,2.10,9.21,5.46,981.98,210.22
3,2024-01-04,1.18,8.17,2.52,991.51,309.00
4,2024-01-05,0.36,8.18,2.28,1001.11,314.89


## 4: Export Aggregated Data
- Save result as National_Irish_Aggregated_Weather_2024.csv in /data/  
 

In [30]:
# Define output path
output_path = "../data/National_Irish_Aggregated_Weather_2024.csv"

# Save the DataFrame as a CSV file
avg_data.to_csv(output_path, index=False)

print(f"Aggregated weather data saved to: {output_path}")

Aggregated weather data saved to: ../data/National_Irish_Aggregated_Weather_2024.csv


## 5: Load Raw Solar Generation Data (15 minute granularity)

In [14]:
import pandas as pd

# Load solar generation data
solar_df = pd.read_csv('../data/National_SolarGeneration_Qtr_Hourly.csv')

# Load Aggregated national weather data for later merge
weather_df = pd.read_csv('../data/National_Irish_Aggregated_Weather_2024.csv')

weather_df.head()
solar_df.head()

,date,solargen
0,1/01/2024 0:00,0.10
1,1/01/2024 0:15,0.10
2,1/01/2024 0:30,0.10
3,1/01/2024 0:45,0.09
4,1/01/2024 1:00,0.10


## 6: Convert 15 min granularity to daily form
- Group solargen by day and sum
- Create new compatible daily_solar dataframe 

In [17]:
# Convert 'date' column to datetime (including time)
solar_df['date'] = pd.to_datetime(solar_df['date'], dayfirst=True)

# Create a 'day' column (date without time) for grouping
solar_df['day'] = solar_df['date'].dt.date

# Sum solar generation by day
daily_solar = solar_df.groupby('day')['solargen'].sum().round(2).reset_index()

# Rename 'day' column to 'date' for merging
daily_solar.rename(columns={'day': 'date'}, inplace=True)

# Convert 'date' back to datetime (without time)
daily_solar['date'] = pd.to_datetime(daily_solar['date'])

daily_solar.head()

,date,solargen
0,2024-01-01,471.02
1,2024-01-02,601.80
2,2024-01-03,1286.11
3,2024-01-04,2788.48
4,2024-01-05,2966.48


## Merge Solar Generation data with national Irish aggregated weather
- Merge final cleaned data  
- Save result as National_Irish_Aggregated_Weather_Solar_2024.csv in /cleaned_data/

In [18]:
# Covert Weather_df columns to datetime format for merge compatibility
weather_df['date'] = pd.to_datetime(weather_df['date'], dayfirst=True)

# Merge with Daily
merged_final = pd.merge(weather_df, daily_solar, on='date', how='left')

# Save final .csv to /Cleaned Data/
merged_final.to_csv('../Cleaned Data/Cleaned_National_Irish_Weather_Solar_2024.csv', index=False)

print(merged_final.head())


        date   rain  maxtp  mintp      cbl  glorad  solargen
0 2024-01-01  11.93  10.57   3.03   981.90   64.44    471.02
1 2024-01-02   5.77  10.22   6.53   973.32  125.44    601.80
2 2024-01-03   2.10   9.21   5.46   981.98  210.22   1286.11
3 2024-01-04   1.18   8.17   2.52   991.51  309.00   2788.48
4 2024-01-05   0.36   8.18   2.28  1001.11  314.89   2966.48
